In [127]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [128]:
import warnings
warnings.filterwarnings('ignore')


In [129]:
import pandas as pd


In [130]:
df = pd.read_csv("clean_air_quality_data.csv")
df.head()

,from_date,to_date,pm25,pm10,no2,nox,nh3,so2,ozone,benzene,toluene,temperature,wind_speed,wind_direction,pressure
0,2019-06-20 14:00:00,2019-06-20 15:00:00,33.00,119.5,23.38,18.15,30.5,7.57,186.32,0.40,64.00,33.97,0.62,172.75,741.0
1,2019-06-20 15:00:00,2019-06-20 16:00:00,33.00,120.0,23.38,18.15,30.5,7.10,191.27,0.40,66.65,33.78,0.62,172.75,741.0
2,2019-06-20 16:00:00,2019-06-20 17:00:00,34.50,117.0,23.38,18.15,30.5,12.00,196.90,0.43,61.30,33.88,0.62,172.75,741.0
3,2019-06-20 17:00:00,2019-06-20 18:00:00,41.67,138.0,23.38,18.15,30.5,12.87,188.53,0.45,62.55,30.75,0.62,172.75,741.0
4,2019-06-20 18:00:00,2019-06-20 19:00:00,46.00,143.0,23.38,18.15,30.5,6.53,168.06,0.35,71.60,29.85,0.62,172.75,741.0


In [131]:
df.dtypes

from_date          object
to_date            object
pm25              float64
pm10              float64
no2               float64
nox               float64
nh3               float64
so2               float64
ozone             float64
benzene           float64
toluene           float64
temperature       float64
wind_speed        float64
wind_direction    float64
pressure          float64
dtype: object

In [132]:
df['from_date'] = pd.to_datetime(df['from_date'])
df['to_date'] = pd.to_datetime(df['to_date'])

In [133]:
df.dtypes

from_date         datetime64[ns]
to_date           datetime64[ns]
pm25                     float64
pm10                     float64
no2                      float64
nox                      float64
nh3                      float64
so2                      float64
ozone                    float64
benzene                  float64
toluene                  float64
temperature              float64
wind_speed               float64
wind_direction           float64
pressure                 float64
dtype: object

In [134]:
df = df.sort_values('from_date')
df = df.set_index('from_date')


In [135]:
df = df.drop(columns=['to_date'])


In [136]:
df.isna().sum().sum()

0

In [138]:
df['pm25_lag_3']  = df['pm25'].shift(3)
df['pm25_lag_6']  = df['pm25'].shift(6)
df['pm25_lag_24'] = df['pm25'].shift(24)  # same hour yesterday


In [139]:
df = df.dropna()


In [140]:
df['pm25_roll_6']  = df['pm25'].rolling(6).mean()
df['pm25_roll_24'] = df['pm25'].rolling(24).mean()


In [141]:
# X_ts = df.drop(columns=['pm25'])
# y_ts = df['pm25']


In [142]:
# split_date = pd.to_datetime("2022-01-01")

# X_train = X_ts[X_ts.index < split_date]
# X_test  = X_ts[X_ts.index >= split_date]

# y_train = y_ts[y_ts.index < split_date]
# y_test  = y_ts[y_ts.index >= split_date]


In [143]:
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

# 1. Initialize the TimeSeriesSplit
# n_splits=5 means it will test the model on 5 different time windows
tscv = TimeSeriesSplit(n_splits=5)

# We use the features and target you defined in your notebook
# Assuming X and y are already prepared from your preprocessing
X = df[trained_features]
y = df['pm25']

mae_scores = []
r2_scores = []

print("🔄 Starting Time-Series Cross-Validation...")

for train_index, test_index in tscv.split(X):
    # Split the data based on time indices
    X_train_cv, X_test_cv = X.iloc[train_index], X.iloc[test_index]
    y_train_cv, y_test_cv = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the model (using XGBoost as an example)
    model_cv = xgb.XGBRegressor(n_estimators=100, learning_rate=0.05)
    model_cv.fit(X_train_cv, y_train_cv)
    
    # Predict and Evaluate
    preds = model_cv.predict(X_test_cv)
    mae_scores.append(mean_absolute_error(y_test_cv, preds))
    r2_scores.append(r2_score(y_test_cv, preds))

# 2. Show the results
print(f"\n✅ Validation Complete")
print(f"Average MAE: {np.mean(mae_scores):.2f} (+/- {np.std(mae_scores):.2f})")
print(f"Average R2 Score: {np.mean(r2_scores):.4f}")

if np.std(r2_scores) > 0.1:
    print("⚠️ Warning: Your model performance varies significantly between time periods.")
else:
    print("💎 Stable Model: Performance is consistent across different time windows.")

🔄 Starting Time-Series Cross-Validation...

✅ Validation Complete
Average MAE: 12.81 (+/- 4.82)
Average R2 Score: 0.8563
💎 Stable Model: Performance is consistent across different time windows.


In [144]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, r2_score
import xgboost as xgb
import numpy as np

tscv = TimeSeriesSplit(n_splits=5)

mae_scores = []
r2_scores = []

X_train_full = X_ts[X_ts.index < split_date]
y_train_full = y_ts[y_ts.index < split_date]



for train_index, val_index in tscv.split(X_train_full):
    X_train_cv, X_val_cv = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train_cv, y_val_cv = y_train_full.iloc[train_index], y_train_full.iloc[val_index]
    
    model_cv = xgb.XGBRegressor(n_estimators=100, learning_rate=0.05)
    model_cv.fit(X_train_cv, y_train_cv)
    
    preds = model_cv.predict(X_val_cv)
    mae_scores.append(mean_absolute_error(y_val_cv, preds))
    r2_scores.append(r2_score(y_val_cv, preds))

print(f"Average MAE: {np.mean(mae_scores):.2f}")
print(f"Average R2 Score: {np.mean(r2_scores):.4f}")

Average MAE: 13.05
Average R2 Score: 0.8217


In [145]:
rf_ts = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

rf_ts.fit(X_train, y_train)


,n_estimators,200
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [146]:
y_pred = rf_ts.predict(X_test)


In [147]:
print("MAE :", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R2  :", r2_score(y_test, y_pred))


MAE : 14.77124611263736
RMSE: 30.552481541500228
R2  : 0.8207671941970055


In [148]:
import joblib
joblib.dump(rf_ts, "pm25_random_forest_model11.pkl")


['pm25_random_forest_model11.pkl']

In [26]:
pip install xgboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 3.2 MB/s  0:00:003.2 MB/s eta 0:00:01:01
Note: you may need to restart the kernel to use updated packages.


In [149]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [150]:
xgb_model = xgb.XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1
)


In [151]:
xgb_model.fit(X_train, y_train)


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [152]:
y_pred_xgb = xgb_model.predict(X_test)


In [153]:
mae  = mean_absolute_error(y_test, y_pred_xgb)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
r2   = r2_score(y_test, y_pred_xgb)

print("XGBoost MAE :", mae)
print("XGBoost RMSE:", rmse)
print("XGBoost R2  :", r2)


XGBoost MAE : 14.020803511639436
XGBoost RMSE: 26.49240971439282
XGBoost R2  : 0.8652379997177512


In [154]:
import joblib
joblib.dump(xgb_model, "pm25_XGBoost_model.pkl")

['pm25_XGBoost_model.pkl']

In [21]:
pip install lightgbm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.0 MB/s  0:00:00m 7.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [155]:
import os
os.environ["MPLBACKEND"] = "Agg"


In [156]:
import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [157]:
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_test  = lgb.Dataset(X_test, label=y_test, reference=lgb_train)


In [158]:
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'seed': 42,
    'verbosity': -1
}


In [159]:
lgb_model = lgb.train(
    params,
    lgb_train,
    num_boost_round=500,
    valid_sets=[lgb_train, lgb_test],
    valid_names=['train', 'test'],
    callbacks=[lgb.early_stopping(stopping_rounds=50)]
)


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[211]	train's rmse: 11.4226	test's rmse: 26.2085


In [160]:
y_pred_lgb = lgb_model.predict(
    X_test,
    num_iteration=lgb_model.best_iteration
)


In [161]:
mae  = mean_absolute_error(y_test, y_pred_lgb)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_lgb))
r2   = r2_score(y_test, y_pred_lgb)

print("LightGBM MAE :", mae)
print("LightGBM RMSE:", rmse)
print("LightGBM R2  :", r2)


LightGBM MAE : 14.226826757768206
LightGBM RMSE: 26.20852820871861
LightGBM R2  : 0.8681106315402574


In [162]:
import pandas as pd

feature_importance = pd.Series(
    lgb_model.feature_importance(),
    index=X_train.columns
).sort_values(ascending=False)

feature_importance.head(10)


pm25_roll_6     1043
pm25_lag_3       901
pm10             769
pm25_lag_6       511
ozone            347
toluene          337
benzene          314
so2              293
pm25_roll_24     266
no2              245
dtype: int32

In [163]:
df['pm25_t+1']  = df['pm25'].shift(-1)
df['pm25_t+6']  = df['pm25'].shift(-6)
df['pm25_t+24'] = df['pm25'].shift(-24)



In [164]:
trained_features = X_train.columns.tolist()



In [169]:
last_row = X_ts[trained_features].iloc[-5]


In [173]:
last_row = X_ts[trained_features].iloc[-7]

next_pm25 = lgb_model.predict(last_row)
print("Next-hour PM2.5 forecast:", next_pm25)


Next-hour PM2.5 forecast: [14.09970783]


In [167]:
def forecast_next_n_steps(model, last_rows_df, feature_columns, n_steps=24):
    history = last_rows_df.copy()
    
    forecasts = []
    
    for i in range(n_steps):
        current_features = history.iloc[-1:][feature_columns]
        
        prediction = model.predict(current_features)[0]
        forecasts.append(prediction)
        

        next_row = history.iloc[-1:].copy()
        
        next_row['pm25'] = prediction 
        
        history = pd.concat([history, next_row])
        
        history['pm25_lag_3'] = history['pm25'].shift(3)
        history['pm25_lag_6'] = history['pm25'].shift(6)
        history['pm25_lag_24'] = history['pm25'].shift(24)
        
        history['pm25_roll_6'] = history['pm25'].rolling(6).mean()
        history['pm25_roll_24'] = history['pm25'].rolling(24).mean()
        
    return forecasts

trained_features = X_train.columns.tolist()
last_rows = df.iloc[-30:].copy() 

pm25_forecast_24h = forecast_next_n_steps(
    xgb_model, 
    last_rows, 
    trained_features, 
    n_steps=10
)

print(pm25_forecast_24h)

[9.693799, 9.245341, 10.187014, 9.335554, 9.682444, 9.776441, 7.752247, 8.394193, 8.394193, 8.394193]


In [171]:
import joblib
joblib.dump(xgb_model, 'air_quality_xgb_model1.pkl')

['air_quality_xgb_model1.pkl']